# 📚 Propuesta de Trabajo: Sistema RAG con Base Vectorial sobre programas de asignaturas de la UNLu
Se propone la implementación de un sistema de Recuperación Aumentada por Generación (RAG) utilizando como colección de documentos los Programas de distintas asignaturas de carreras de la Universidad Nacional de Luján.

El sistema se construirá mediante el uso de embeddings, el motor de búsqueda vectorial Chroma, y un modelo de lenguaje de Llama para generar respuestas precisas basadas en la información recuperada.

## 📘 Introducción: LlamaIndex y RAG

### 🧠 ¿Qué es LlamaIndex?
**LlamaIndex** es un framework diseñado para construir aplicaciones basadas en modelos de lenguaje (LLM) que utilizan **información externa y específica de dominio**.  
Permite extender las capacidades de los LLMs integrando datos privados, actualizados o especializados.

### 🔎 ¿Qué es Retrieval-Augmented Generation (RAG)?
Los modelos de lenguaje tradicionales tienen un conocimiento limitado a los datos públicos con los que fueron entrenados.

La **Generación Aumentada por Recuperación (RAG)** soluciona este problema incorporando información relevante y dinámica desde fuentes externas (documentos, bases de datos, APIs, etc.), justo en el momento de la consulta.

Este enfoque representa un cambio importante:
- Las respuestas **no dependen solo del conocimiento entrenado** en el modelo.
- Se incorpora **contexto recuperado en tiempo real**, lo que mejora precisión y reduce alucinaciones.

### ⚙️ Flujo del proceso con LlamaIndex + RAG

1. 🗣️ **Consulta del Usuario**  
   El usuario formula una pregunta o solicitud.

2. 📂 **Recuperación de Contexto**  
   El sistema consulta un índice previamente construido y selecciona los fragmentos de texto más relevantes.

3. 🧩 **Integración de Datos**  
   Se pueden combinar múltiples fuentes:  
   - Datos estructurados (bases SQL, CSVs)  
   - Datos no estructurados (PDFs, documentos)  
   - Datos programáticos (APIs)

4. ✍️ **Construcción del Prompt**  
   Se genera un prompt enriquecido que contiene la pregunta original y los fragmentos recuperados como contexto adicional.

5. 🤖 **Generación de la Respuesta**  
   El LLM utiliza el contexto aportado para generar una respuesta precisa, sin depender únicamente de su memoria interna.

6. 📬 **Entrega de la Respuesta**  
   El modelo devuelve una respuesta contextualizada, combinando su conocimiento general con la información específica recuperada.


# 🛠️ Pipeline RAG implementado con LlamaIndex

## 📦 Instalación de paquetes y configuración de API Key

In [ ]:
%pip install llama-index llama-index-embeddings-jinaai llama-index-vector-stores-chroma llama-index-llms-huggingface-api --quiet
%pip install requests beautifulsoup4 pdf2image PyPDF2 pytesseract --quiet
!sudo apt install tesseract-ocr poppler-utils
!sudo apt update
!sudo apt install tesseract-ocr-spa

## 📥 Loading de Documentos


### 🧾 ¿Qué se carga en LlamaIndex?
Los documentos pueden provenir de **cualquier fuente de datos**, como: PDFs, Sitios web, Bases de datos o APIs

### 🔌 Readers
LlamaIndex utiliza componentes llamados **Readers** (también conocidos como *Loaders* o *Connectors*) que:

- ✅ Permiten importar información desde múltiples formatos.
- 📚 Soportan datos: **Estructurados** (como tablas o bases SQL), **No estructurados** (como texto libre, PDFs, HTMLs) y **Programáticos** (como respuestas desde APIs).
Transforman las fuentes originales en objetos `Document`, que contienen:
   - El **contenido textual extraído**.
   - **Metadatos** como nombre de archivo, URL, título, etc.

In [ ]:
import os, requests, tempfile, shutil, pytesseract
from pathlib import Path
from bs4 import BeautifulSoup
from pdf2image import convert_from_path
from PyPDF2 import PdfReader
from urllib.parse import urljoin
from llama_index.core import SimpleDirectoryReader
import re

def download_pdf(url):
    response = requests.get(url)
    response.raise_for_status()
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    with open(tmp_file.name, "wb") as f:
        f.write(response.content)
    return tmp_file.name

def is_pdf_selectable(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text = page.extract_text()
            if text and text.strip():
                return True
        return False
    except Exception as e:
        print(f"⚠️ Error leyendo PDF: {e}")
        return False

def ocr_pdf_to_text(pdf_path):
    images = convert_from_path(pdf_path)
    text = ""
    for img in images:
        text += pytesseract.image_to_string(img, lang="spa") + "\n"
    return text

def process_pdf_from_url(url, file_name, output_dir="."):
    pdf_path = download_pdf(url)

    base_name = Path(file_name).stem  # Ej: "11071_0"

    output_txt_path = os.path.join(output_dir, f"{base_name}.txt")
    output_pdf_path = os.path.join(output_dir, file_name)

    if is_pdf_selectable(pdf_path):
        print("✅ PDF con texto seleccionable. Guardando sin modificar...")
        shutil.move(pdf_path, output_pdf_path)
        print(f"📄 PDF guardado en: {output_pdf_path}")
        return output_pdf_path
    else:
        print("🧾 PDF escaneado. Usando OCR y guardando como texto...")
        text = ocr_pdf_to_text(pdf_path)
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"📝 Texto OCR guardado en: {output_txt_path}")
        return output_txt_path

def get_meta(file_path):
    return {
        "carrera": os.path.basename(os.path.dirname(file_path)),
        "asignatura": os.path.splitext(os.path.basename(file_path))[0]
    }

def get_programas_dir(url, name):
    PROGRAMAS_DIR = "./programas/" + name
    return PROGRAMAS_DIR

def get_programas_de_asignatura(url):
    PAGINA_ASIGNATURA = url

    try:
        resp = requests.get(PAGINA_ASIGNATURA)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"❌ Error al obtener la página: {e}")
        exit()

    nombre_carrera = soup.find_all("h1", class_="page-title")[0].get_text(strip=True)
    nombre_carrera = re.sub(r"\s*\(.*?\)", "", nombre_carrera).strip()
    print(f"Nombre de la carrera: {nombre_carrera}")

    PROGRAMAS_DIR = get_programas_dir(url, nombre_carrera)

    os.makedirs(PROGRAMAS_DIR, exist_ok=True)

    links = soup.find_all("a", href=True)
    pdf_links = []

    for link in links:
        href = link["href"]
        if href.lower().endswith(".pdf") and "/Programas/" in href:
            full_url = urljoin(PAGINA_ASIGNATURA, href)
            texto = link.get_text(strip=True)  # también podés usar link.text.strip()
            pdf_links.append((full_url, texto))

    pdf_links = sorted(set(pdf_links))
    print(f"Enlaces PDF detectados: {len(pdf_links)}")
    print()

    for url,file_name in pdf_links:
        print(f"Nombre de la asignatura: {file_name}")
        process_pdf_from_url(url, file_name, PROGRAMAS_DIR)
        print()

    reader_programas = SimpleDirectoryReader(PROGRAMAS_DIR, file_metadata = get_meta)
    return reader_programas.load_data(), nombre_carrera

def es_url_valida(url):
    patron = r"^https://www\.certificaciones\.unlu\.edu\.ar/\?q=node/\d+$"
    return re.match(patron, url) is not None


documentos_dict = {}

entrada = ""
while entrada != "SALIR":
    print("Las listas de programas de asignaturas tienen esta forma: 'https://www.certificaciones.unlu.edu.ar/?q=node/<X>'. X varía dependiendo de la carrera")
    entrada = input("Ingrese la página web que lista las asignaturas: ")

    if es_url_valida(entrada):
        documentos_programa, nombre_carrera = get_programas_de_asignatura(entrada)
        documents_dict[nombre_carrera] = documentos_programa
    elif entrada.upper() != "SALIR":
        print("URL inválida. Intente nuevamente.")
print("Saliendo...")

# EJEMPLO DE URL A INGRESAR: https://www.certificaciones.unlu.edu.ar/?q=node/43

#### Si se cuenta ya con los archivos textuales de los programas en el entorno de ejecución, se puede ejecutar esta celda para no procesar nuevamente desde la página oficial de la asignatura

In [ ]:
import os

PROGRAMAS_DIR = "programas"

def get_meta(file_path):
    return {
        "carrera": os.path.basename(os.path.dirname(file_path)),
        "asignatura": os.path.splitext(os.path.basename(file_path))[0]
    }

def cargar_programas_por_carrera():
    documentos_por_carrera = {}
    for nombre_carrera in os.listdir(PROGRAMAS_DIR):
        ruta_carrera = os.path.join(PROGRAMAS_DIR, nombre_carrera)
        if os.path.isdir(ruta_carrera):
            reader_programas = SimpleDirectoryReader(ruta_carrera, file_metadata=get_meta)
            documentos = reader_programas.load_data()
            documentos_por_carrera[nombre_carrera] = documentos
    return documentos_por_carrera

documentos_dict = cargar_programas_por_carrera()

### Fragmentación y Nodos
Luego de obtener los objetos `Document`, estos son divididos en **Nodos**, que son las **unidades mínimas de información**, fragmentos pequeños y optimizados para búsqueda y recuperación en el sistema RAG.


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

def clean_text(text):
    return text.encode("utf-8", "ignore").decode("utf-8")

# Definir splitter
splitter = SentenceSplitter(chunk_size=2048, chunk_overlap=500, include_metadata=True)

# Obtener diccionario de nodos por carrera
nodos_dict = {}
for nombre_carrera, documentos in documentos_dict.items():
    nodos_dict[nombre_carrera] = splitter.get_nodes_from_documents(documentos)
    for nodo in nodos_dict[nombre_carrera]:
        nodo.text = clean_text(nodo.text)

## 🧭 Indexing (Indexación de Documentos)

### 🧠 ¿Qué es el Indexado?
La indexación consiste en convertir los documentos cargados en **vectores de embeddings** (representaciones numéricas del significado de cada Nodo) y organizarlos en una estructura especial llamada **índice vectorial** *(Vector Store)*, que incluye también los metadatos asociados.

> 📌 Cada Node del documento se transforma en un vector de embedding.

### 🔍 ¿Para qué sirve el índice?

- Permite buscar por **significado**, no solo por coincidencia exacta.
- Devuelve los **fragmentos más relevantes** para una consulta.
- Es la base para construir el **contexto que se le pasa al LLM** (en RAG).

> 📌 El índice es esencial para recuperar la información correcta en tiempo real y enriquecer las respuestas del modelo con datos específicos.

In [ ]:
from llama_index.embeddings.jinaai import JinaEmbedding

# Definir embedding
embed_model = JinaEmbedding(
    model_name="jina-embeddings-v2-base-en",
    api_key="INGRESE_SU_API_KEY"
)

## 💾 Storing (Almacenamiento de Embeddings)

### 🧠 ¿Dónde se almacenan los embeddings?
Una vez generados los embeddings de cada fragmento (Node), estos pueden guardarse en una **base de datos vectorial** como: **Chroma**, **FAISS**, **Weaviate**, **Pinecone**, entre otros

Esto permite que el sistema sea **persistente** y **escalable**, sin tener que recalcular embeddings cada vez que se ejecuta.

### 📦 ¿Qué se almacena?

En el proceso de almacenamiento, se guarda:
- 📊 **Embeddings**: el vector que representa el significado del nodo.
- 🏷️ **Metadatos**: como nombre del archivo, página, sección, título, etc.
- 🔗 **Relación con el documento original**: para reconstruir fácilmente el contexto completo.

### 🧰 ¿Para qué sirve almacenar los embeddings?

El almacenamiento en vector stores permite:

- ✅ **Evitar recomputar** embeddings en cada ejecución.
- 🔁 **Reutilizar el índice** en distintos procesos o notebooks.
- 🚀 **Realizar consultas eficientes** sobre grandes volúmenes de información.
- 🧠 **Mantener contexto** para el modelo sin procesar los documentos nuevamente.

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb, os, unicodedata, re

def normalizar_nombre(nombre):
    nombre = unicodedata.normalize('NFKD', nombre).encode('ascii', 'ignore').decode('ascii')
    nombre = nombre.lower().replace(" ", "_")
    nombre = re.sub(r"[^a-z0-9._-]", "", nombre)
    nombre = re.sub(r"^[^a-z0-9]+", "", nombre)
    nombre = re.sub(r"[^a-z0-9]+$", "", nombre)
    return nombre

def create_index_from_documents(nodes, collection_name):
    os.makedirs("./chroma_db", exist_ok=True)
    db = chromadb.PersistentClient(path="./chroma_db")
    chroma_collection = db.get_or_create_collection(collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex(
        nodes,
        embed_model=embed_model,
        vector_store=vector_store
    )
    return index

indexes = {}

# Obtener diccionario de indices por carrera
for nombre_carrera, nodes in nodos_dict.items():
    nombre_index = normalizar_nombre(f"programa_{nombre_carrera}")
    indexes[nombre_carrera] = create_index_from_documents(nodes, nombre_index)

## 🔎 Querying (Consulta y Generación de Respuestas)


En esta etapa, el sistema utiliza múltiples componentes para **recuperar información relevante** desde el índice y generar una **respuesta contextualizada** con ayuda del LLM.

### 🧭 1. Retriever: Buscar Nodos Relevantes

Un **Retriever** básico se encarga de:

1. 🔢 Convertir la consulta del usuario en un **embedding**.
2. 📍 Buscar los **Nodes más cercanos** (más relevantes) en el espacio vectorial del índice.
3. 📦 Devolver esos fragmentos como **contexto** para el modelo.

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.tools import QueryEngineTool

# CON HUGGING FACE VIA API
HF_TOKEN = "INGRESE_SU_API_KEY"

# Definimos el llm a utilizar como base para el RAG
llm = HuggingFaceInferenceAPI(
    model_name="meta-llama/Llama-3.3-70B-Instruct",
    token=HF_TOKEN,
    provider="hf-inference"
)

# Definimos las query_engine_tools, que contienen dentro los Retrievers
tools = []
for nombre_carrera, index in indexes.items():
    query_engine = index.as_query_engine(
        llm=llm,
        response_mode="tree_summarize",
        use_async=True
    )
    tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f"Información sobre los programas de las asignaturas de la carrera de {nombre_carrera} de la Universidad Nacional de Lujan. Incluye contenidos, equipo docente, condicion de regular y aprobado y correlatividad de cada materia/asignatura"
    )
    tools.append(tool)

### 🧠 2. Router: Elegir el Índice Correcto

Trabajamos con **múltiples fuentes de conocimiento** (En este caso: programas de asignaturas de distintas carreras), por lo que podemos usar un `Router`.

📌 **¿Qué hace?**

- Evalúa la consulta.
- Selecciona automáticamente el **Retriever** (y por lo tanto el índice) más adecuado.
- Redirige la consulta al índice más relevante.

> Útil cuando tenés múltiples dominios o tipos de documentos en un mismo sistema.


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector

# Creamos el router con un selector que elige solo uno de los Retriever especificados
router = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(llm=llm),
    query_engine_tools=tools,
    llm=llm
)

consulta = input("Ingrese su consulta: ")

response = router.query(consulta)
print("💬 Con RAG:\n", response)
print()

llm_noRAG = HuggingFaceInferenceAPI(
    model_name="meta-llama/Llama-3.3-70B-Instruct",
    token=HF_TOKEN,
    provider="hf-inference"
)
response = llm_noRAG.complete(consulta)
print("💬 Sin RAG:\n", response)
print()